In [231]:
import pandas as pd
import joblib
import yaml

In [232]:
industry = pd.read_excel('../Final/Data/korean_industry.xlsx', dtype = {'코드':str})
SMEs = pd.read_csv('../Data/original/Regional_specialization_SMEs.csv', encoding='cp949')
Factories = pd.read_csv('../Final/Data/Factory_total.csv')
Factories = Factories[Factories['상태'] == '진행'] ## 현재 진행중인 데이터만 가져오기

In [233]:
Factories.columns

Index(['매물위치(1)', '매물위치(2)', '매물위치(3)', '종류/장점', '제목', '상태', '가격', '면적', '방식',
       '참고', '조회수', '매물위치(url)'],
      dtype='object')

In [234]:
local_governments = {
    '1': '서울특별시', 
    '2': '부산광역시', 
    '3': '대구광역시', 
    '4': '인천광역시', 
    '5': '광주광역시', 
    '6': '대전광역시', 
    '7': '울산광역시', 
    '8': '세종특별자치시', 
    '9': '경기도', 
    '10': '강원특별자치도', 
    '11': '충청북도', 
    '12': '충청남도', 
    '13': '전북특별자치도', 
    '14': '전라남도', 
    '15': '경상북도', 
    '16': '경상남도', 
    '17': '제주특별자치도'
}

In [235]:
local_governments_english = {
    '서울특별시': 'Seoul',
    '부산광역시': 'Busan',
    '대구광역시': 'Daegu',
    '인천광역시': 'Incheon',
    '광주광역시': 'Gwangju',
    '대전광역시': 'Daejeon',
    '울산광역시': 'Ulsan',
    '세종특별자치시': 'Sejong',
    '경기도': 'Gyeonggi',
    '강원특별자치도': 'Gangwon',
    '충청북도': 'Chungbuk',
    '충청남도': 'Chungnam',
    '전북특별자치도': 'Jeonbuk',
    '전라남도': 'Jeonnam',
    '경상북도': 'Gyeongbuk',
    '경상남도': 'Gyeongnam',
    '제주특별자치도': 'Jeju'
}

In [236]:
# local_goverment 선택
print("공장 부지 선정을 위한 지자체를 선택해주세요.")
for key, value in local_governments.items():
    print(f"{key}. {value}")

user_input_local_goverment = input("지자체를 선택해주세요: ")
if user_input_local_goverment in local_governments:
    korean_name = local_governments[user_input_local_goverment]
    english_name = local_governments_english[korean_name]
    print(f'선택한 지자체는 "{korean_name}" 입니다\n적합한 모델을 불러오는 중입니다. 잠시만 기다려주세요.')
    
    # 데이터 로드
    data = pd.read_csv(f'../Final/Data/{english_name}_encoding_data.csv')
    
    # 모델 로드
    lgbm_best_model = joblib.load(f'../Final/Final_Model/{english_name}_best_model.pkl')
    label_encoders = joblib.load(f'../Final/Final_Model/{english_name}_label_encoders_lgbm.pkl')
    original_values = joblib.load(f'../Final/Final_Model/{english_name}_original_values_lgbm.pkl')
    # 대표업종 라벨 인코더 로드
    label_encoder_industry = label_encoders['대표업종']

    # 데이터프레임의 최빈값 계산
    most_frequent_values = data.mode().iloc[0].astype(int)

    print("모델 및 인코딩 준비 완료")
else:
    raise ValueError("Invalid input for local_goverment. Please enter a number from the list.")

공장 부지 선정을 위한 지자체를 선택해주세요.
1. 서울특별시
2. 부산광역시
3. 대구광역시
4. 인천광역시
5. 광주광역시
6. 대전광역시
7. 울산광역시
8. 세종특별자치시
9. 경기도
10. 강원특별자치도
11. 충청북도
12. 충청남도
13. 전북특별자치도
14. 전라남도
15. 경상북도
16. 경상남도
17. 제주특별자치도
선택한 지자체는 "경기도" 입니다
적합한 모델을 불러오는 중입니다. 잠시만 기다려주세요.
모델 및 인코딩 준비 완료


In [237]:
most_frequent_values

공장구분            0
설립구분            4
입주형태            0
보유구분            1
등록구분            4
남자종업원           3
여자종업원           0
외국인남자종업원        0
외국인여자종업원        0
종업원합계           5
공장규모            1
용도지역            1
지목              0
용지면적            0
제조시설면적        198
부대시설면적          0
건축면적          198
지식산업센터명       269
대표업종          432
공장주소_지번     19099
도               0
시              30
구              79
Name: 0, dtype: int32

In [238]:
# 3. 입력받지 못한 값, 대표 업종 코드에 대한 최빈값으로 결측치를 채움
mode_values_by_business = data.groupby('대표업종').agg(lambda x: x.mode().iloc[0])
most_frequent_representative_business = data['대표업종'].mode().iloc[0] #대표업종 최빈값
most_frequent_representative_business_encoder = str(label_encoder_industry.inverse_transform([most_frequent_representative_business])[0])

In [239]:
# 사용자 입력 데이터 받기
user_input_보유구분 = input("어떤 공장을 보유하고 싶으신가요? (임대: 1 / 자가: 2): ")
if user_input_보유구분 == '1':
    user_input_보유구분 = '임대'
elif user_input_보유구분 == '2':
    user_input_보유구분 = '자가'
else:
    raise ValueError("Invalid input for 보유구분. Please enter 1 or 2.")

user_input_등록구분 = input("공장 등록구분을 선택해주세요.(1: 등록변경 / 2: 부분등록 / 3: 신규등록 / 4: 완료신고 )")
등록구분_dict = {'1': '등록변경', '2': '부분등록', '3': '신규등록', '4': '완료신고'}
if user_input_등록구분 in 등록구분_dict:
    user_input_등록구분 = 등록구분_dict[user_input_등록구분]
else:
    raise ValueError("Invalid input for 등록구분. Please enter 1, 2, 3, or 4.")

user_input_공장규모 = input("공장 규모를 선택해주세요.(1: 소기업 / 2: 중기업 / 3: 대기업 )")
공장규모_dict = {'1': '소기업', '2': '중기업', '3': '대기업'}
if user_input_공장규모 in 공장규모_dict:
    user_input_공장규모 = 공장규모_dict[user_input_공장규모]
else:
    raise ValueError("Invalid input for 공장규모. Please enter 1, 2, or 3.")

user_input_남자종업원 = int(input("남자종업원 수를 입력해주세요: "))
user_input_여자종업원 = int(input("여자종업원 수를 입력해주세요: "))
user_input_외국인남자종업원 = int(input("외국인 남자종업원 수를 입력해주세요: "))
user_input_외국인여자종업원 = int(input("외국인 여자종업원 수를 입력해주세요: "))

# 종업원합계 자동 계k산
user_input_종업원합계 = user_input_남자종업원 + user_input_여자종업원 + user_input_외국인남자종업원 + user_input_외국인여자종업원

user_input_대표업종 = int(input("대표업종 코드를 입력해주세요. (예: 18111): "))

In [240]:
# 최빈값 계산
most_frequent_representative_business_encoder = data['대표업종'].mode()[0]

In [241]:
most_frequent_representative_business_encoder

432

In [242]:
# 대표업종 코드 검증 및 대체
def get_industry_name_or_default(user_input_code, industry_df, label_encoder, default_code):
    user_input_code_str = str(user_input_code)
    if user_input_code_str in industry_df['코드'].values:
        entry_value = industry_df.loc[industry_df['코드'] == user_input_code_str, '항목명'].values[0]
        return user_input_code_str, entry_value
    else:
        print(f"입력한 대표업종 코드({user_input_code})가 데이터에 존재하지 않습니다.")
        default_code_str = str(default_code)
        default_name = industry_df.loc[industry_df['코드'] == default_code_str, '항목명'].values[0]
        print(f"대표업종 코드를 최빈값인 {default_code_str} ({default_name})으로 대체합니다.")
        return default_code_str, default_name

# 최빈값 가져오기
most_frequent_representative_business = data['대표업종'].mode().iloc[0]
most_frequent_representative_business_code = str(label_encoder_industry.inverse_transform([most_frequent_representative_business])[0])

# 사용자 입력 코드 검증 및 대체
final_대표업종_code, final_대표업종_name = get_industry_name_or_default(user_input_대표업종, industry, label_encoder_industry, most_frequent_representative_business_code)


In [243]:
most_frequent_representative_business_encoder

432

In [244]:
industry['코드']

0       01110
1       01121
2       01122
3       01123
4       01131
        ...  
1192    97000
1193    98100
1194    98200
1195    99001
1196    99009
Name: 코드, Length: 1197, dtype: object

# user_input에 제조면적도 추가하기!?

In [245]:
# 사용자 입력 데이터
user_input = {
    '보유구분': user_input_보유구분, 
    '등록구분': user_input_등록구분, 
    '공장규모': user_input_공장규모, 
    '남자종업원': user_input_남자종업원, 
    '여자종업원': user_input_여자종업원, 
    '외국인남자종업원': user_input_외국인남자종업원, 
    '외국인여자종업원': user_input_외국인여자종업원, 
    '종업원합계': user_input_종업원합계,
    '대표업종': final_대표업종_code
}

In [246]:
encoded_user_input = {}

for col, value in user_input.items():
    if col == '대표업종':
        # 대표업종에 대해 라벨 인코딩 적용
        try:
            encoded_user_input[col] = label_encoder_industry.transform([value])[0]
        except ValueError:
            print(f"대표업종 코드({value})가 라벨 인코더에 존재하지 않습니다. 최빈값으로 대체합니다.")
            encoded_user_input[col] = label_encoder_industry.transform([most_frequent_representative_business_code])[0]
    elif col in label_encoders:
        # 해당 컬럼에 대해 라벨 인코딩 적용
        encoded_user_input[col] = label_encoders[col].transform([value])[0]
    else:
        encoded_user_input[col] = value

print(encoded_user_input)

{'보유구분': 0, '등록구분': 1, '공장규모': 1, '남자종업원': 1, '여자종업원': 1, '외국인남자종업원': 1, '외국인여자종업원': 1, '종업원합계': 4, '대표업종': 101}


In [247]:
# 결측치 채우기 (최빈값으로)
for col in data.columns:
    if col not in encoded_user_input:
        if col in most_frequent_values.index:
            encoded_user_input[col] = most_frequent_values[col]

# DataFrame 형태로 변환
input_df = pd.DataFrame([encoded_user_input])

# '구' 열 제거
if '구' in input_df.columns:
    input_df = input_df.drop(columns=['구'])


In [248]:
print(input_df)

   보유구분  등록구분  공장규모  남자종업원  여자종업원  외국인남자종업원  외국인여자종업원  종업원합계  대표업종  공장구분  ...  \
0     0     1     1      1      1         1         1      4   101     0  ...   

   용도지역  지목  용지면적  제조시설면적  부대시설면적  건축면적  지식산업센터명  공장주소_지번  도   시  
0     1   0     0     198       0   198      269    19099  0  30  

[1 rows x 22 columns]


In [249]:
model_features = lgbm_best_model.booster_.feature_name()

In [250]:
model_features

['공장구분',
 '설립구분',
 '입주형태',
 '보유구분',
 '등록구분',
 '남자종업원',
 '여자종업원',
 '외국인남자종업원',
 '외국인여자종업원',
 '종업원합계',
 '공장규모',
 '용도지역',
 '지목',
 '용지면적',
 '제조시설면적',
 '부대시설면적',
 '건축면적',
 '지식산업센터명',
 '대표업종']

In [251]:
encoded_user_input = {}

# 입력 데이터에 모든 피처를 포함하도록 수정
for col in model_features:
    if col not in encoded_user_input:
        encoded_user_input[col] = most_frequent_values[col] if col in most_frequent_values.index else 0

input_df = pd.DataFrame([encoded_user_input])
input_df = input_df[model_features]

In [252]:
input_df = pd.DataFrame([encoded_user_input])

In [253]:
# 모델이 학습할 때 사용한 열 순서로 정렬
model_features = ['공장구분', '설립구분', '입주형태', '보유구분', '등록구분', '남자종업원', '여자종업원', 
                  '외국인남자종업원', '외국인여자종업원', '종업원합계', '공장규모', '용도지역', '지목', 
                  '용지면적', '제조시설면적', '부대시설면적', '건축면적', '지식산업센터명', '대표업종']

input_df = input_df[model_features]

In [254]:
input_df

,공장구분,설립구분,입주형태,보유구분,등록구분,남자종업원,여자종업원,외국인남자종업원,외국인여자종업원,종업원합계,공장규모,용도지역,지목,용지면적,제조시설면적,부대시설면적,건축면적,지식산업센터명,대표업종
0,0,4,0,1,4,3,0,0,0,5,1,1,0,0,198,0,198,269,432


In [255]:
# 인코딩된 값을 원래 값으로 변환
decoded_user_input = {}
for col in input_df.columns:
    if col in label_encoders:
        decoded_user_input[col] = label_encoders[col].inverse_transform(input_df[col])[0]
    else:
        decoded_user_input[col] = input_df[col].values[0]

# 결과 출력
decoded_df = pd.DataFrame([decoded_user_input])
print(decoded_df)

  공장구분 설립구분 입주형태 보유구분  등록구분  남자종업원  여자종업원  외국인남자종업원  외국인여자종업원  종업원합계 공장규모  \
0   개별   일반        자가  완료신고      3      0         0         0      5  소기업   

                  용도지역    지목  용지면적  제조시설면적  부대시설면적  건축면적 지식산업센터명   대표업종  
0  관리지역/계획관리지역/계획관리지역/  공장용지     0     198       0   198    해당없음  29294  


In [256]:
# 4. 모델 추론 수행
prediction = lgbm_best_model.predict(input_df)

In [257]:
# 결과를 원래 값으로 변환
predicted_label = prediction[0]
predicted_class = original_values['구'][predicted_label]
print(f"모델 예측 결과: {predicted_class}")

모델 예측 결과: 정남면


### SMEs 주력산업 제공

In [258]:
# 주력산업 찾기
user_input_local_goverment = int(user_input_local_goverment)
selected_industry = SMEs[SMEs['연번'] == user_input_local_goverment]['주력산업'].values[0]

# 결과 출력
print(f"선택한 지자체의 주력산업은: {selected_industry}")

선택한 지자체의 주력산업은: 중소벤처 지역특화 업종이 존재하지 않습니다.


## OPENAI 통해서 산업 장단점, 전망 파악하기

In [259]:
#config.yaml 파일 경로
config_path = '../Config/config.yaml'

#yaml 파일 열기
with open(config_path, 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)

Service_key = config['ServiceKey']['OpenAIkEY']

In [260]:
from openai import OpenAI, OpenAIError, RateLimitError
import time
import backoff

# OpenAI 클라이언트 초기화
client = OpenAI(
    api_key = Service_key 
)

# 재시도 로직을 사용한 함수
@backoff.on_exception(backoff.expo, (OpenAIError, RateLimitError), max_tries=5)
def get_completion(**kwargs):
    return client.chat.completions.create(**kwargs)

message_content = f"업종코드 {final_대표업종_name}에 관련해서 장점, 단점, 전망, 결론에 대해서 사업을 시작하기 위한 사람들을 위해 자세하게 설명해줘."


# 요청 데이터 설정
data = {
    'model': 'gpt-4o',  # 최신 모델 사용
    'messages': [{'role': 'user', 'content': message_content}],
    'max_tokens': 1200,
    'temperature': 0.7  # 응답의 창의성 조절
}

try:
    response = get_completion(**data)
    print(response.choices[0].message.content.strip())
except RateLimitError as e:
    print("Rate limit exceeded. Please wait and try again later.")
    time.sleep(60)  # 1분 대기 후 재시도
except OpenAIError as e:
    print(f"An error occurred: {e}")

가방 및 기타 보호용 케이스 제조업은 다양한 종류의 가방과 케이스를 제작하는 산업입니다. 이 산업은 패션, 여행, 전자기기 보호 등 다양한 목적을 위한 제품을 포함하며, 소비자의 수요에 따라 끊임없이 변화합니다. 이제 이 업종의 장점, 단점, 전망, 결론에 대해 자세히 설명해드리겠습니다.

### 장점
1. **다양한 시장 기회**: 가방 및 케이스 제조업은 패션, 여행, 전자기기 보호 등 다양한 시장에서 기회를 제공합니다. 각 시장마다 특화된 제품을 제공할 수 있어 다변화를 통해 리스크를 분산할 수 있습니다.

2. **높은 부가가치**: 브랜드와 디자인이 중요한 요소로 작용하여 고부가가치 제품을 생산할 수 있습니다. 소비자들은 특별한 디자인이나 브랜드 가치에 대해 기꺼이 더 많은 비용을 지불할 의향이 있습니다.

3. **글로벌 시장 접근성**: 전자상거래와 국제 물류의 발달로 인해 글로벌 시장에 쉽게 접근할 수 있습니다. 다양한 국가의 소비자들을 대상으로 제품을 판매할 수 있습니다.

4. **기술 발전**: 3D 프린팅, 스마트 소재 등 최신 기술을 활용해 혁신적인 제품을 개발할 수 있는 여지가 많습니다. 이는 시장에서의 경쟁력을 높이는 데 도움이 됩니다.

### 단점
1. **높은 초기 투자 비용**: 제조업 특성상 공장 설비, 원자재 구매, 인력 고용 등 초기 투자 비용이 높습니다. 이는 자금 조달에 부담이 될 수 있습니다.

2. **경쟁 심화**: 이미 많은 기업들이 존재하고 있으며, 특히 패션 가방 시장에서는 경쟁이 매우 치열합니다. 저가 제품부터 고가 제품까지 다양한 경쟁자들과 싸워야 합니다.

3. **소비자 취향 변화**: 패션 업계의 특성상 소비자 취향이 빠르게 변화합니다. 이에 따라 제품 개발과 생산 계획을 유연하게 조정해야 합니다.

4. **품질 관리**: 다양한 소재와 디자인을 사용하기 때문에 품질 관리가 복잡할 수 있습니다. 특히 고가의 제품일수록 품질 문제가 발생하면 브랜드 이미지에 큰 타격을 입을 수 있습니다.

### 전

In [261]:
predicted_class

'정남면'

In [262]:
korean_name

'경기도'

In [263]:
# 조건에 맞는 행 필터링
filtered_factories = Factories[
    (Factories['매물위치(1)'] == korean_name) & 
    (Factories['매물위치(3)'] == predicted_class)
]

# 필요한 열만 선택하여 필터링된 데이터프레임 생성
columns_to_display = ['매물위치(1)', '매물위치(2)', '매물위치(3)', '종류/장점', '제목', '상태', '가격', '면적', '방식', '참고', '조회수', '매물위치(url)']
filtered_factories_to_display = filtered_factories[columns_to_display]

# 필터링된 데이터가 있는지 확인
if filtered_factories_to_display.empty:
    print("현재 매물이 존재하지 않습니다. 선택한 지자체 관련된 모든 공장 데이터를 제공합니다.")
    filtered_factories = Factories[
        (Factories['매물위치(1)'] == korean_name)
    ]
    filtered_factories_to_display = filtered_factories[columns_to_display]
    
# DataFrame을 Jupyter Notebook 내에서 보기 좋게 출력
display(filtered_factories_to_display.style.set_properties(**{'text-align': 'left'}).set_table_styles({
    '매물위치(1)': [{'selector': 'th', 'props': [('text-align', 'left')]}],
    '매물위치(2)': [{'selector': 'th', 'props': [('text-align', 'left')]}],
    '매물위치(3)': [{'selector': 'th', 'props': [('text-align', 'left')]}],
    '종류/장점': [{'selector': 'th', 'props': [('text-align', 'left')]}],
    '제목': [{'selector': 'th', 'props': [('text-align', 'left')]}],
    '상태': [{'selector': 'th', 'props': [('text-align', 'left')]}],
    '가격': [{'selector': 'th', 'props': [('text-align', 'left')]}],
    '면적': [{'selector': 'th', 'props': [('text-align', 'left')]}],
    '방식': [{'selector': 'th', 'props': [('text-align', 'left')]}],
    '참고': [{'selector': 'th', 'props': [('text-align', 'left')]}],
    '조회수': [{'selector': 'th', 'props': [('text-align', 'left')]}],
    '매물위치(url)': [{'selector': 'th', 'props': [('text-align', 'left')]}]
}))

# 결과를 CSV 파일로 저장
filtered_factories_to_display.to_csv(f'{korean_name}_{predicted_class}_filtered_factories.csv', index=False, encoding='cp949')

,매물위치(1),매물위치(2),매물위치(3),종류/장점,제목,상태,가격,면적,방식,참고,조회수,매물위치(url)
15,경기도,화성시,정남면,시설-38 재활용/폐기물,"화성 정남면 PP, PE 재활용공장 파쇄, 분쇄, 압출 공장...",진행,"167,000만원","대지 : 1,580㎡ (478평) 건물 : 715㎡ (216평)",매매,구분 층명 구조 용도 면적(㎡) 지상 일동 1층 경량철골조 판넬 분뇨...,119,http://www.okfactory.com/main/estat_view.php?board_data=aWR4PTcyOTAmc3RhcnRQYWdlPTEyJmxpc3RObz01MTk2JnRhYmxlPSZjb2RlPSZzZWFyY2hfaXRlbT0mc2VhcmNoX29yZGVyPSZzZWFyY2hfbm89||&search_items=dHJhZGVfdHlwZT0mXyZ0cmFkZV9pbWc9Jl8mYXJlYV9pbWc9Jl8mc2VhcmNoX29yZGVyPSZfJnNlYXJjaF9vcGVuPSZfJnNlYXJjaF9ubz0mXyZzZWFyY2hfY29tYXBhbnk9Jl8mc2VhcmNoX25hbWU9Jl8mc2VhcmNoX2FkZHI9Jl8mc2VhcmNoX2Jsb2NrPSZfJnNlYXJjaF9sb3Q9Jl8mc2VhcmNoX29wZW49Jl8mdW5zaW5nY29kZTE9MTE4MTYzNjA4NSZfJnVuc2luZ2NvZGUyPTEyMTM3MDAxMTMmXyZ1bnNpbmdjb2RlMz0mXyZkaXNwbGF5X2ltZz0mXyZ1c2VyaWQ9Jl8mcHllb25nMT0mXyZweWVvbmcyPSZfJnB5b25nMT0mXyZweW9uZzI9Jl8mbW9uZXkxPSZfJm1vbmV5Mj0mXyZob2lzdDE9Jl8maG9pc3QyPSZfJnBvd2VyMT0mXyZwb3dlcjI9Jl8maGVpZ2h0MT0mXyZoZWlnaHQyPSZfJmNpdHk9Jl8mZ29vPSZfJmRvbmc9Jl8mc2l6ZXR5cGU9Jl8mc29ydD0=||
38,경기도,화성시,정남면,공장-29.30 기계장비/자동차,화성시 정남면 일반산업단지 신축공장 임대. 전체 대지 5322..,진행,"8,400만원","대지 : 17,600㎡ (5,324평) 건물 : 13,890㎡ (4,202평)",월세,"신축공장 입니다. 공장 2개 동 구성. 각동별 2100평 ,..",119,http://www.okfactory.com/main/estat_view.php?board_data=aWR4PTcyNjYmc3RhcnRQYWdlPTM2Jmxpc3RObz01MTczJnRhYmxlPSZjb2RlPSZzZWFyY2hfaXRlbT0mc2VhcmNoX29yZGVyPSZzZWFyY2hfbm89||&search_items=dHJhZGVfdHlwZT0mXyZ0cmFkZV9pbWc9Jl8mYXJlYV9pbWc9Jl8mc2VhcmNoX29yZGVyPSZfJnNlYXJjaF9vcGVuPSZfJnNlYXJjaF9ubz0mXyZzZWFyY2hfY29tYXBhbnk9Jl8mc2VhcmNoX25hbWU9Jl8mc2VhcmNoX2FkZHI9Jl8mc2VhcmNoX2Jsb2NrPSZfJnNlYXJjaF9sb3Q9Jl8mc2VhcmNoX29wZW49Jl8mdW5zaW5nY29kZTE9MTE4MTYzNjA4NSZfJnVuc2luZ2NvZGUyPTEyMTM3MDAxMTMmXyZ1bnNpbmdjb2RlMz0mXyZkaXNwbGF5X2ltZz0mXyZ1c2VyaWQ9Jl8mcHllb25nMT0mXyZweWVvbmcyPSZfJnB5b25nMT0mXyZweW9uZzI9Jl8mbW9uZXkxPSZfJm1vbmV5Mj0mXyZob2lzdDE9Jl8maG9pc3QyPSZfJnBvd2VyMT0mXyZwb3dlcjI9Jl8maGVpZ2h0MT0mXyZoZWlnaHQyPSZfJmNpdHk9Jl8mZ29vPSZfJmRvbmc9Jl8mc2l6ZXR5cGU9Jl8mc29ydD0=||
124,경기도,화성시,정남면,공장-16.33 기타소형/일반,화성 정남 교통요충지 최고의 소형공장 매매[부지 210평+도로..,진행,"85,000만원",대지 : 714㎡ (216평) 건물 : 199㎡ (60평),매매,지목 공장용지 면적 694㎡ 외 도로지분 약 8..,350,http://www.okfactory.com/main/estat_view.php?board_data=aWR4PTcxNzUmc3RhcnRQYWdlPTEyMCZsaXN0Tm89NTA4MiZ0YWJsZT0mY29kZT0mc2VhcmNoX2l0ZW09JnNlYXJjaF9vcmRlcj0mc2VhcmNoX25vPQ==||&search_items=dHJhZGVfdHlwZT0mXyZ0cmFkZV9pbWc9Jl8mYXJlYV9pbWc9Jl8mc2VhcmNoX29yZGVyPSZfJnNlYXJjaF9vcGVuPSZfJnNlYXJjaF9ubz0mXyZzZWFyY2hfY29tYXBhbnk9Jl8mc2VhcmNoX25hbWU9Jl8mc2VhcmNoX2FkZHI9Jl8mc2VhcmNoX2Jsb2NrPSZfJnNlYXJjaF9sb3Q9Jl8mc2VhcmNoX29wZW49Jl8mdW5zaW5nY29kZTE9MTE4MTYzNjA4NSZfJnVuc2luZ2NvZGUyPTEyMTM3MDAxMTMmXyZ1bnNpbmdjb2RlMz0mXyZkaXNwbGF5X2ltZz0mXyZ1c2VyaWQ9Jl8mcHllb25nMT0mXyZweWVvbmcyPSZfJnB5b25nMT0mXyZweW9uZzI9Jl8mbW9uZXkxPSZfJm1vbmV5Mj0mXyZob2lzdDE9Jl8maG9pc3QyPSZfJnBvd2VyMT0mXyZwb3dlcjI9Jl8maGVpZ2h0MT0mXyZoZWlnaHQyPSZfJmNpdHk9Jl8mZ29vPSZfJmRvbmc9Jl8mc2l6ZXR5cGU9Jl8mc29ydD0=||
279,경기도,화성시,정남면,토지-공장.창고부지,화성시 정남면 별장지 공장용지.창고용지.야적장.연수원. 요양원..,진행,"228,200만원","대지 : 4,713㎡ (1,426평) 건물 : 199㎡ (60평)",매매,"""총면적 1426평"" 토지의 모양을 살리고 지가를 상승시키기 위해 본..",419,http://www.okfactory.com/main/estat_view.php?board_data=aWR4PTcwMTMmc3RhcnRQYWdlPTI4OCZsaXN0Tm89NDkyMSZ0YWJsZT0mY29kZT0mc2VhcmNoX2l0ZW09JnNlYXJjaF9vcmRlcj0mc2VhcmNoX25vPQ==||&search_items=dHJhZGVfdHlwZT0mXyZ0cmFkZV9pbWc9Jl8mYXJlYV9pbWc9Jl8mc2VhcmNoX29yZGVyPSZfJnNlYXJjaF9vcGVuPSZfJnNlYXJjaF9ubz0mXyZzZWFyY2hfY29tYXBhbnk9Jl8mc2VhcmNoX25hbWU9Jl8mc2VhcmNoX2FkZHI9Jl8mc2VhcmNoX2Jsb2NrPSZfJnNlYXJjaF9sb3Q9Jl8mc2VhcmNoX29wZW49Jl8mdW5zaW5nY29kZTE9MTE4MTYzNjA4NSZfJnVuc2luZ2NvZGUyPTEyMTM3MDAxMTMmXyZ1bnNpbmdjb2RlMz0mXyZkaXNwbGF5X2ltZz0mXyZ1c2VyaWQ9Jl8mcHllb25nMT0mXyZweWVvbmcyPSZfJnB5b25nMT0mXyZweW9uZzI9Jl8mbW9uZXkxPSZfJm1vbmV5Mj0mXyZob2lzdDE9Jl8maG9pc3QyPSZfJnBvd2VyMT0mXyZwb3dlcjI9Jl8maGVpZ2h0MT0mXyZoZWlnaHQyPSZfJmNpdHk9Jl8mZ29vPSZfJmRvbmc9Jl8mc2l6ZXR5cGU9Jl8mc29ydD0=||
368,경기도,화성시,정남면,공장-27.28 정밀/전기장비,"IT, 전자부품 업종, 연구소/사옥빌딩(공장) 매각 [대지 54..",진행,"380,000만원","대지 : 1,796㎡ (543평) 건물 : 639㎡ (193평)",매매,"◆ ARCHITECTURE IT, 반도체, 벤처사옥, 카페, 스크린골프,..",755,http://www.okfactory.com/main/es

In [264]:
import openai
print(openai.__version__)

1.34.0
